In [1]:
# -*- coding: utf-8 -*-
"""
Created on Sun Aug 16 17:10:53 2020

@author: wanxiang.shen@u.nus.edu
"""

import warnings, os
warnings.filterwarnings("ignore")


import pandas as pd
import numpy as np

from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score
from sklearn.metrics import auc as calculate_auc

import matplotlib.pyplot as plt
import seaborn as sns

from aggmap import AggMap, AggModel, loadmap
from aggmap import show
np.random.seed(666) #just for reaptable results

def prc_auc_score(y_true, y_score):
    precision, recall, threshold  = precision_recall_curve(y_true, y_score) #PRC_AUC
    auc = calculate_auc(recall, precision)
    return auc


def score(dfr):
    y_true = dfr.y_true
    y_score = dfr.y_score
    y_pred = dfr.y_score.round()

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    acc = (tp + tn) / sum([tn, fp, fn, tp])
    auc = roc_auc_score(y_true, y_score)

    print('acc: %.3f, roc-auc: %.3f' % (acc,auc))

    return acc, auc

In [2]:
label = pd.read_excel('./data/subject_info.xlsx', index_col = 'Subject_ID')
data = pd.read_excel('./data/data_species_7278_preprocessed.xlsx', index_col = 0)

In [3]:
label.groupby('Group').size()

Group
HS               30
Healthy         127
MP               40
Stage_0          27
Stage_III_IV     54
Stage_I_II       69
dtype: int64

In [4]:
mp = loadmap('./CRC_stage.mp')

## Model-1: Healthy vs. MP:

In [5]:
stage = 'Stage_I_II'
model_dir = './%s_model' % stage

sld = label[label.Group.isin(['Healthy', stage])].index
dfy = label.loc[sld][['Group']]
dfx = data.loc[sld]
dfx = np.log(dfx + 1e-8)
dfy.shape, dfx.shape

((196, 1), (196, 7278))

In [6]:
X = mp.batch_transform(dfx.values, scale_method = 'standard', fillnan = np.log(1e-8)) # NaN values should be the lowest value
Y = pd.get_dummies(dfy['Group']).values

100%|##########| 196/196 [00:26<00:00,  7.33it/s]


In [7]:
dfy.shape, dfx.shape

((196, 1), (196, 7278))

In [8]:
class_ = pd.get_dummies(dfy['Group']).columns
class_

Index(['Healthy', 'Stage_I_II'], dtype='object')

In [9]:
outer_fold = 10
repeat_seeds = [8, 16, 32, 64, 128]#5 repeats random seeds 8, 16, 32, 64, 128

each_fold_results = []
run_all_res = []
for repeat_seed in repeat_seeds: 
    
    outer = StratifiedKFold(n_splits = outer_fold, shuffle = True, random_state = repeat_seed)
    outer_idx = outer.split(X, Y[:,0])

    
    run_one_res = []
    for i, idx in enumerate(outer_idx):
        
        fold_num = "fold_%s" % str(i).zfill(2) 
        
        train_idx, test_idx = idx
        
        testY = Y[test_idx]
        testX = X[test_idx]
        
        trainX = X[train_idx]
        trainY = Y[train_idx]

        print("\n input train and test X shape is %s, %s " % (trainX.shape,  testX.shape))
        
        clf = AggModel.MultiClassEstimator(epochs = 50, gpuid = 3, conv1_kernel_size = 13,
                                           batch_size = 2, verbose = 0) #
        clf.fit(trainX, trainY ) 

        ## save model for interpretation
        if repeat_seed == 8:
            clf._model.save(os.path.join(model_dir, 'seed_%s_fold_%s.h5' % (repeat_seed, i)))
    
        y_true = testY[:,1] #Stage
        y_score = clf.predict_proba(testX)[:,1]
        
        dfr = pd.DataFrame([y_true, y_score]).T
        dfr.columns = ['y_true', 'y_score']
        dfr.index = dfy.iloc[test_idx].index
        
        acc, auc = score(dfr)
        
        run_one_res.append(dfr)
        
        each_fold_results.append([auc, i, repeat_seed])
        
    run_all_res.append(pd.concat(run_one_res))


 input train and test X shape is (176, 86, 85, 5), (20, 86, 85, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=13, epochs=50, gpuid='3')
acc: 0.850, roc-auc: 0.967

 input train and test X shape is (176, 86, 85, 5), (20, 86, 85, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=13, epochs=50, gpuid='3')
acc: 0.850, roc-auc: 0.802

 input train and test X shape is (176, 86, 85, 5), (20, 86, 85, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=13, epochs=50, gpuid='3')
acc: 0.850, roc-auc: 0.978

 input train and test X shape is (176, 86, 85, 5), (20, 86, 85, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=13, epochs=50, gpuid='3')
acc: 0.850, roc-auc: 0.901

 input train and test X shape is (176, 86, 85, 5), (20, 86, 85, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=13, epochs=50, gpuid='3')
acc: 0.800, roc-auc: 0.813

 input train and test X shape is (176, 86, 85, 5), (20, 86, 85, 5) 
MultiClassEstimator(batch_size=2, conv1_kernel_size=13, epoch

In [10]:
results = []
for dfr, repeat_seed in zip(run_all_res,repeat_seeds):

    y_true = dfr.y_true
    y_score = dfr.y_score
    y_pred = dfr.y_score.round()


    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    acc = (tp + tn) / sum([tn, fp, fn, tp])

    sensitivity = tp / sum([tp, fn])
    specificity = tn / sum([tn, fp])

    prc_auc = prc_auc_score(y_true, y_score)
    roc_auc = roc_auc_score(y_true, y_score)

    precision = tp / sum([tp, fp])
    recall =  tp / sum([tp, fn]) #equals to sensitivity

    res = {
           'repeat_seed':repeat_seed,

           'accuracy':acc, 

           'prc_auc':prc_auc, 
           'roc_auc':roc_auc,

           'sensitivity': sensitivity, 
           'specificity': specificity,

           'precision':precision,
           'recall':recall,
           'F1': 2*precision*sensitivity/(precision+sensitivity)
          }
    
    results.append(res)

In [11]:
pd.DataFrame(results).roc_auc.mean()

0.8527216706607327

In [12]:
pd.DataFrame(results)

,repeat_seed,accuracy,prc_auc,roc_auc,sensitivity,specificity,precision,recall,F1
0,8,0.826531,0.834333,0.853018,0.594203,0.952756,0.872340,0.594203,0.706897
1,16,0.836735,0.834965,0.859865,0.608696,0.960630,0.893617,0.608696,0.724138
2,32,0.806122,0.812909,0.843661,0.536232,0.952756,0.860465,0.536232,0.660714
3,64,0.816327,0.839082,0.856100,0.608696,0.929134,0.823529,0.608696,0.700000
4,128,0.826531,0.824098,0.850964,0.579710,0.960630,0.888889,0.579710,0.701754


In [13]:
pd.DataFrame(results).to_csv('./results/performance_results_%s.csv' % stage)

In [14]:
pd.DataFrame(each_fold_results,columns = ['ROC-AUC', 'fold', 'seed']).to_csv('./results/each_fold_performance_results_%s.csv' % stage)